In [2]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import fileinput
import pprint
import math
import numpy as np
import pandas as pd
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer 
from sklearn.metrics import f1_score
from yake import KeywordExtractor
from keybert import KeyBERT


In [3]:
f = pd.read_csv("Train.csv")
f.head()

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


In [5]:
df2 = f.sample(frac=0.00006)
df2

,Id,Title,Body,Tags
596664,596665,parse a csv file that contains commans in the ...,<p>i have to use awk to print out 4 different ...,csv awk
4844623,4844624,removeEventListener seems to not remove the li...,<p>I wrote this actionscript code:</p>\n\n<pre...,flex event-handling
4629230,4629231,"Representing a 2D map of doubles in as few ""pa...",<p>I am working on a turn-based-game AI using ...,artificial-intelligence neural-network game-ai...
2066057,2066058,Particle engine problem,<p>i am implementing particle engine and here ...,iphone cocos2d cocos2d-iphone particles
1472228,1472229,Control WP7 and WP8 app with bluetooth headset,<p>I have developed a music player app for Win...,c# windows-phone-7 audio bluetooth windows-pho...
...,...,...,...,...
3841323,3841324,How to remotely view activity logs from a Java...,<p>I'd like to remotely monitor the activity o...,java-ee logging glassfish vaadin
3092335,3092336,Background image Magnifying/Zooming in iPhone ...,<p>I have been making an app over the past few...,iphone-5
1343385,1343386,How to salvage a lvm/luks install from custom ...,<p>I was battling with setting up a Mint insta...,luks dm-crypt
734545,734546,Android adjustResize - How to disable the fade...,<p>When using <strong>android:windowSoftInputM...,android phonegap webview android-webview


In [7]:
first10k = f.iloc[:362,:]
first10k

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents
...,...,...,...,...
357,358,When clicking on the view the animation kicks ...,<p><strong>(This topic is not about .setfillaf...,android animation
358,359,Set a new origin of PDFView's documentView,<p>I'm trying to make a custom version of PDFV...,objective-c osx cocoa subclass pdfview
359,360,Array RGB manipulation,<p>Lets say I have an <code>int[][] arrayA</co...,java arrays rgb
360,361,re-start animation with start button with burs...,<p>I have created a simple animation with burs...,javascript html html5 canvas


In [9]:
# ps = PorterStemmer()

# for i in range(first10k.shape[0]):
#     if i == 2000:
#         break
#     text = first10k['Title'][i] + first10k['Tags'][i]
#     fromTags = first10k['Tags'][i].split(" ")

#     temp = []
#     for anstag in fromTags:
#         if("-" in anstag):
#             temp += anstag.split("-")
#         else:
#             temp.append(anstag)
     
#     temp = [ps.stem(w) for w in temp] 
#     fromTags = temp
            
#     newTags = [] 
#     for tag in fromTags:
#         if tag in text:
#             newTags.append(tag)
#     if i == 1576:
#         print(text)
#         print(fromTags)
#         print(newTags)

    
    

In [10]:
randomsample = df2.reset_index(level=0)
randomsample

,index,Id,Title,Body,Tags
0,596664,596665,parse a csv file that contains commans in the ...,<p>i have to use awk to print out 4 different ...,csv awk
1,4844623,4844624,removeEventListener seems to not remove the li...,<p>I wrote this actionscript code:</p>\n\n<pre...,flex event-handling
2,4629230,4629231,"Representing a 2D map of doubles in as few ""pa...",<p>I am working on a turn-based-game AI using ...,artificial-intelligence neural-network game-ai...
3,2066057,2066058,Particle engine problem,<p>i am implementing particle engine and here ...,iphone cocos2d cocos2d-iphone particles
4,1472228,1472229,Control WP7 and WP8 app with bluetooth headset,<p>I have developed a music player app for Win...,c# windows-phone-7 audio bluetooth windows-pho...
...,...,...,...,...,...
357,3841323,3841324,How to remotely view activity logs from a Java...,<p>I'd like to remotely monitor the activity o...,java-ee logging glassfish vaadin
358,3092335,3092336,Background image Magnifying/Zooming in iPhone ...,<p>I have been making an app over the past few...,iphone-5
359,1343385,1343386,How to salvage a lvm/luks install from custom ...,<p>I was battling with setting up a Mint insta...,luks dm-crypt
360,734545,734546,Android adjustResize - How to disable the fade...,<p>When using <strong>android:windowSoftInputM...,android phonegap webview android-webview


In [14]:
def process(text):
    ps = PorterStemmer() 
    text = text.lower()
    text = re.sub(r'[^\w\s\\\+]', ' ', text)
    text = re.sub("\d+", " ", text)
    tokens = word_tokenize(str(text)) 
    tokens = [ps.stem(w) for w in tokens]
    stop_words = stopwords.words('english')
    stop_words.append("www")
    stop_words.append("http")
    result = []
    for token in tokens:
        if not token.isdigit() and not token in stop_words and not token in string.punctuation  :
            result.append(token)
    return result 

In [16]:
regex = "(?<=<p>)(.|\n)*?(?=<\/p>)|(?<=<li>)(.|\n)*?(?=<\/li>)"

In [20]:
idf = {}
tokens = []
#texts = []
for i in range(randomsample.shape[0]):
    #if i % 1000 == 0:
        #print(i)
    tokensForThisDoc = process(randomsample["Title"][i] ) #+ process(randomsample["Title"][i] )
    for match in re.finditer(regex, randomsample["Body"][i]):
        text = (str) (match.group(0))
        text = process(text)
        tokensForThisDoc += text
    tokens.append(tokensForThisDoc)
    
print(len(tokens))
        

362


In [23]:
stackOverFlowDict = {}
for tok in tokens:
    for w in set(tok):
        if w in stackOverFlowDict:
             stackOverFlowDict[w]+= 1
        else:
             stackOverFlowDict[w] = 1
#stackOverFlowDict['file']

In [24]:
for w in stackOverFlowDict:
    idf[w]= np.log(len(tokens)/stackOverFlowDict[w])
#print(stackOverFlowDict['file'])

In [25]:
tf = []
cleanedTexts = []
first10ktokens = []
from collections import Counter
for i in range(first10k.shape[0]):
    #if i % 1000 == 0:
         #print(i)
    tokensForThisDoc =  process(first10k["Title"][i] ) + process(first10k["Title"][i] )
    for match in re.finditer(regex, first10k["Body"][i]):
        text = (str) (match.group(0))
        text = process(text)
        tokensForThisDoc += text
    first10ktokens.append(tokensForThisDoc)
    cleanedTexts.append(' '.join((tokensForThisDoc)))
    
for tok in first10ktokens:
    tf.append(Counter(tok))


362
9
1.60118477067738


In [27]:
ranked = []

for i in range(len(tf)):
    rankedWordsForThisDoc = []
    for w in tf[i]:
        idfValue = 0.0001
        if w in idf:
            idfValue = idf[w]
        num = tf[i][w] *  idfValue
        tfidf = (w, num )
        rankedWordsForThisDoc.append(tfidf)
    rankedWordsForThisDoc.sort(key = lambda x: x[1], reverse = True)
    ranked.append(rankedWordsForThisDoc)
#pprint.pprint(ranked[0])
#pprint.pprint(first10k['Tags'][0])

[('upload', 22.87321602087561),
 ('file', 14.41066293609642),
 ('type', 13.780749979483106),
 ('check', 12.979036729107209),
 ('imag', 9.461134748838438),
 ('apart', 5.8916442118257715),
 ('gif', 5.198497031265826),
 ('without', 5.048696763678594),
 ('jpg', 4.505349850705881),
 ('extens', 4.282206299391671),
 ('matter', 4.099884742597716),
 ('g', 3.8122026701459353),
 ('png', 3.183594010723561),
 ('e', 3.058430867769555),
 ('give', 3.0012724539296065),
 ('text', 2.9472052326593308),
 ('php', 2.7561499958966214),
 ('anoth', 2.4904468301636156),
 ('someth', 2.2807262991815467),
 ('chang', 2.063002815336676),
 ('problem', 1.4971950571533323),
 ('way', 1.4971950571533323),
 ('use', 1.4085168119700324),
 ('like', 1.2666713985415),
 ('mime', 0.0004),
 ('jpeg', 0.0001),
 ('bmp', 0.0001),
 ('uploadifi', 0.0001),
 ('octal', 0.0001)]
'php image-processing file-upload upload mime-types'


In [29]:
vectorizer = TfidfVectorizer(use_idf = True, max_df = 0.5, min_df = 1, ngram_range = (1,1))

In [30]:
print(len(cleanedTexts))
print(cleanedTexts[0])

362
check upload file imag without mime type check upload file imag without mime type like check upload file imag file e g png jpg jpeg gif bmp anoth file problem use uploadifi upload file chang mime type give text octal someth mime type matter file type upload way check upload file imag apart check file extens use php


In [31]:
vectors = vectorizer.fit_transform(cleanedTexts)

In [32]:
dict_of_tokens={i[1]:i[0] for i in vectorizer.vocabulary_.items()}


In [33]:
tfidf_vectors = []  # all deoc vectors by tfidf
for row in vectors:
    tfidf_vectors.append({dict_of_tokens[column]:value for (column,value) in zip(row.indices,row.data)})

In [34]:
tfidfSingleKeywords = ranked
tfidfKeyphrases = []
for i in tfidf_vectors:
    exx = list(i.items())
    exx.sort(key = lambda x: x[1], reverse = True)
    tfidfKeyphrases.append(exx)
tfidfKeyphrases[0][:10]

[('upload', 0.5259716104666542),
 ('file', 0.4230630953637333),
 ('mime', 0.41144195169224096),
 ('check', 0.32759108163129486),
 ('type', 0.29543166590435543),
 ('imag', 0.23634533272348432),
 ('without', 0.11817266636174216),
 ('octal', 0.10286048792306024),
 ('uploadifi', 0.10286048792306024),
 ('bmp', 0.10286048792306024)]

In [98]:
topicRankKeyWords = []
for j in range(len(cleanedTexts)):
    if j % 1000 == 0:
        print(j)
    x = keywords.keywords(cleanedTexts[j]).split('\n')
    j = []
    for xx in x:
        j += xx.split(" ")
    topicRankKeyWords.append(j)
topicRankKeyWords[0]

0
['check upload file', 'type', 'use']
['firefox', 'vim', 'ctrl w', 'action']
['matlab file construct', 'error', 'datafram']
['thi', 'object', 'want', 'googl', 'url', 'buri', 'simpl']
['modifi', 'thi']
['applic', 'set proxi', 'use machin']
['coreplot', 'img', 'com', 'p', 'imag', 'draw barplot', 'jpg']
['code', 'like', 'phone app', 'fetch', 'work wp', 'figur']
['net librari gener javascript', 'ani', 'em', 'creat']
['sql server', 'concaten imposs']
['code', 'reflector ildasm', 'obfusc achiev crash', 'protect', 'singl', 'strong']
['random number gener', 'good', 'anyon suggest', 'tree', 'c', 'digit']
['mdaemon postfix']
['manual', 'doe anyon']
['window', 'tab', 'classic']
['like code control', 'friend', 'class', 'user', 'php framework url convent', 'id', 'answer', 'edit', 'want', 'realli']
['creat rd temporari file r', 'like', 'look', 'packag']
['control', 'properti', 'use', 'ani', 'accord']
['play', 'javascript', 'minifi', 'main', 'mainli', 'js', 'github', 'greenscript']
['code', 'string'

['tabl', 'seed statement', 'anyon gener']
['thi command', 'domain window', 'b', 'program', 'net time']
['delet', 'receiv', 'list']
['form', 'item', 'excel file', 'fetch data', 'ani']
['tool', 'custom clean', 'run', 'behavior']
['use getselectionstart', 'want', 'touch']
['excel', 'programm']
['code', 'grid', 'set url']
['exampl', 'overrid', 'method', 'doe', 'whi', 'fix thi', 'problem', 'attribut']
['ga', 'thi', 'com event', 'googl analyt', 'paramet', 'count', 'href', 'thank', 'peopl']
['href', 'repositori', 'thi', 'like', 'tri', 'packagist', 'org']
['attribut option', 'joomla', 'element']
['thi', 'multipl', 'page webbrows']
['code', 'optimis', 'execut', 'queri']
['method', 'thi', 'observ', 'becaus', 'sync', 'object', 'code', 'problem question chang tri', 'declar', 'work', 'time', 'kvo', 'insid']
['app', 'io']
['p', 'class diagram', 'import', 'classa']
['tabl', 'count', 'queri', 'post']
['api']
['project strong', 'work', 'mvc', 'open sourc stock', 'plan', 'notic', 'develop']
['tabl', 'se

['thi', 'manag', 'virtual develop environ enterpris network', 'ha', 'question', 'local control', 'tool', 'struggl', 'product', 'allow', 'purpos exist', 'hardwar', 'class', 'esx', 'access', 'team', 'begin', 'vs', 'separ', 'free', 'type', 'sinc', 'non']
['page', 'html', 'save', 'publish', 'theme', 'contact']
['x', 'integr like thi', 'int_']
['server', 'window', 'thank', 'powershel', 'thi', 'work', 'suggest', 'run']
['applic', 'application', 'destroy']
['code', 'browser', 'page', 'case', 'exampl', 'href', 'boy']
['error pass', 'paramet']
['mercuri', 'work directori', 'dropbox']
['uiwebview', 'sourc', 'work']
['list', 'solut']
['thrown', 'servic', 'loop', 'collect', 'run', 'throw']
['stock', 'perform', 'dll consol applic', 'task']
['code', 'receiv', 'blackberri push', 'url', 'applic', 'subscript handl', 'devic']
['hover', 'block', 'scroll', 'href html', 'mosnadzor']
['messag', 'weblog', 'work', 'properti', 'use spring', 'question', 'thi', 'config', 'file', 'class']
['use', 'xml', 'work', '

['check', 'upload', 'file', 'type', 'use']

In [36]:
len(topicRankKeyWords)

362

In [37]:
kw = KeywordExtractor(lan="en", n=1, top=10)
yakeKeyWords = []
for j in range(len(cleanedTexts)):
    if j % 500 == 0:
        print(j)
    keywords = kw.extract_keywords(text=cleanedTexts[j])
    keywords = [(x,y) for x, y in keywords]
    keywords.sort(key = lambda x: x[1], reverse = True)
    if (len(keywords)==0):
        print(i)
    yakeKeyWords.append(keywords)
yakeKeyWords[0]

0


[('png', 0.10279670715507412),
 ('jpg', 0.10279670715507412),
 ('jpeg', 0.10279670715507412),
 ('php', 0.08649916640114916),
 ('check', 0.0244818306339192),
 ('type', 0.0244818306339192),
 ('imag', 0.023348600683915085),
 ('mime', 0.023348600683915085),
 ('file', 0.014743000453205674),
 ('upload', 0.01467397481700154)]

In [59]:
keyBertWords = []
kw_model = KeyBERT()
for j in range(len(cleanedTexts)):        
    if j % 50 == 0:
        print(j)
    keywords = kw_model.extract_keywords(cleanedTexts[j])
    keyBertWords.append(keywords)
    if j == 0:
        print(keywords)
keyBertWords[0]


0
[('jpeg', 0.4553), ('mime', 0.4379), ('uploadifi', 0.4053), ('jpg', 0.3945), ('png', 0.3698)]
50
100
150
200
250
300
350


[('jpeg', 0.4553),
 ('mime', 0.4379),
 ('uploadifi', 0.4053),
 ('jpg', 0.3945),
 ('png', 0.3698)]

In [60]:
def evaluation(output, answer_key, j):
    
    correct = 0
    output_length = len(output)
    key_length = len(answer_key)
    if len(answer_key) == 0:
        print("problem with:" +str(j))
    for i in output:
        if i in answer_key:
            correct += 1
            
    precision = correct/(output_length + 0.001)
    recall = correct/(key_length + 0.001)
    fmeasure = 0
    if precision != 0 and recall != 0:
        fmeasure = 1/((1/precision)+(1/recall))
    return fmeasure
      #  return f1_score(answer_key, output)
#     print("\tPrecision: %.2f \n\tRecall: %.2f \n\tF-Measure: %.2f" % (precision, recall, fmeasure))

In [61]:
answer_key = []
for i in range(len(first10k)):
    ps = PorterStemmer()
    if i % 500 == 0:
        print(i)
    ans = first10k["Tags"][i].split(' ')
    temp = []
    for anstag in ans:
        if("-" in anstag):
            temp += anstag.split("-")
        else:
            temp.append(anstag)
    temptemp =  [ps.stem(w) for w in temp]
    answer_key.append(list(set(temptemp)))
answer_key[0]

0


['file', 'php', 'type', 'upload', 'process', 'imag', 'mime']

In [41]:
new = ranked[1534][:10]
print(new)
for i in yakeKeyWords[1534][:10]:
    if i[0] in [x[0] for x in ranked[1534]][:10]:
        for j in range(len(new)):
            if new[j][0] == i[0]:
                new[j]= (i[0], new[j][1]*i[1])


print(new)   
print([x[0] for x in new[:10]])
print([x[0] for x in ranked[1534]][:10])
print([x[0] for x in yakeKeyWords[1534]][:10])
print(topicRankKeyWords[1534][:10])
print([x[0] for x in tfidfKeyphrases[1534]][:10])
print(answer_key[1534])
print(first10k['Tags'][1534])
print(first10k['Title'][1534])
print(first10k['Body'][1534])
print("text is:" + cleanedTexts[1534])
print(process(first10k['Title'][1534]))
print(process(first10k['Body'][1534]))

IndexError: list index out of range

In [62]:
keyBertWords[0]

[('jpeg', 0.4553),
 ('mime', 0.4379),
 ('uploadifi', 0.4053),
 ('jpg', 0.3945),
 ('png', 0.3698)]

In [95]:
def newTry(a, b):
    print('new')
    new = a
    for i in b[:10]:
        if i[0] in [x[0] for x in a][:10]:
            for j in range(len(new)):
                if new[j][0] == i[0]:
                    new[j]= (i[0], new[j][1]*i[1])
    print(new)
    new.sort(key = lambda x: x[1], reverse = True)

    return new


def average(lst):
    return sum(lst) / len(lst)                          



In [96]:
# 10 key words
evaluationSelftfidf = []
evalYake = []
evalTopicRank = []
evalTFIDFLib = []
evalkeyBert = []
evalBertYake10 = []
evalBertKey10 = []
evalBertTopic10 = []
evalBertSelf10 = []

# 7 key words
evaluationSelftfidf7 = []
evalYake7 = []
evalTopicRank7 = []
evalTFIDFLib7 = []
evalkeyBert7 = []
evalBertYake7 = []
evalBertKey7 = []
evalBertTopic7 = []
evalBertSelf7 = []




# 5 key words


evaluationSelftfidf5 = []
evalYake5 = []
evalTopicRank5 = []
evalTFIDFLib5 = []
evalkeyBert5 = []
evalBertYake5 = []
evalBertKey5 = []
evalBertTopic5 = []
evalBertSelf5 = []



# 3 key words
evaluationSelftfidf3 = []
evalYake3 = []
evalTopicRank3 = []
evalTFIDFLib3 = []
evalkeyBert3 = []
evalBertYake3 = []
evalBertKey3 = []
evalBertTopic3 = []
evalBertSelf3 = []




# 1 key words

evaluationSelftfidf1 = []
evalYake1 = []
evalTopicRank1 = []
evalTFIDFLib1 = []
evalkeyBert1 = []
evalBertYake1 = []
evalBertKey1 = []
evalBertTopic1 = []
evalBertSelf1 = []



for i in range(len(answer_key)):
  
    if i % 500 == 0:
        print(i)
        
#     print("KeyBert")
#     print(keyBertWords[i])
#     print("Yake")
#     print(yakeKeyWords[i])
    bertYake = [newTry(keyBertWords[i], yakeKeyWords[i]) for i in range(len(ranked))]    
    bertYake[i] = [x[0] for x in bertYake[i]]
    bertYake = list(Counter(bertYake[i]))[:10]
    
    bertKey = [newTry(keyBertWords[i], tfidfKeyphrases[i]) for i in range(len(ranked))]    
    bertKey[i] = [x[0] for x in bertKey[i]]
    bertKey = list(Counter(bertKey[i]))[:10]
    
    bertSelf = [newTry(keyBertWords[i], ranked[i]) for i in range(len(ranked))]    
    bertSelf[i] = [x[0] for x in bertSelf[i]]
    bertSelf = list(Counter(bertSelf[i]))[:10]
    
    bertTopic = [newTry(topicRankKeyWords[i], keyBertWords[i]) for i in range(len(ranked))]    
    bertTopic[i] = [x[0] for x in bertTopic[i]]
    bertTopic = list(Counter(bertTopic[i]))[:10]
    
#     keybertEval = evaluation([x[0] for x in keyBertWords[i]][:10], answer_key[i], i)
#     selfTFIDFEval = evaluation([x[0] for x in ranked[i]][:10], answer_key[i], i)
#     yakeEval = evaluation([x[0] for x in yakeKeyWords[i]][:10], answer_key[i], i)
#     topicRankEval =evaluation(topicRankKeyWords[i][:10], answer_key[i], i)
#     tfidfLibEval = evaluation([x[0] for x in tfidfKeyphrases[i]][:10], answer_key[i],i)
#     print("output")
#     print(allKeyWords)
#     print("answer")
#     print(answer_key[i])
    bertYakeEval = evaluation(bertYake, answer_key[i],i)
    bertKeyEval10 = evaluation(bertKey, answer_key[i],i)
    bertSelfEval10 = evaluation(bertSelf, answer_key[i],i)
#    bertTopicEval10 = evaluation(bertTopic, answer_key[i],i)
    
#     keybertEval7 = evaluation([x[0] for x in keyBertWords[i]][:7], answer_key[i], i)
#     selfTFIDFEval7 = evaluation([x[0] for x in ranked[i]][:7], answer_key[i], i)
#     yakeEval7 = evaluation([x[0] for x in yakeKeyWords[i]][:7], answer_key[i], i)
#     topicRankEval7 =evaluation(topicRankKeyWords[i][:7], answer_key[i], i)
#     tfidfLibEval7 = evaluation([x[0] for x in tfidfKeyphrases[i]][:7], answer_key[i],i)
    bertYakeEval7 = evaluation(bertYake[:7], answer_key[i],i)
    bertKeyEval7 = evaluation(bertKey[:7], answer_key[i],i)
    bertSelfEval7 = evaluation(bertSelf[:7], answer_key[i],i)

#    bertTopicEval7 = evaluation(bertTopic[:7], answer_key[i],i)

    
    
#     keybertEval5 = evaluation([x[0] for x in keyBertWords[i]][:5], answer_key[i], i)
#     selfTFIDFEval5 = evaluation([x[0] for x in ranked[i]][:5], answer_key[i], i)
#     yakeEval5 = evaluation([x[0] for x in yakeKeyWords[i]][:5], answer_key[i], i)
#     topicRankEval5 =evaluation(topicRankKeyWords[i][:5], answer_key[i], i)
#     tfidfLibEval5 = evaluation([x[0] for x in tfidfKeyphrases[i]][:5], answer_key[i],i)
    bertYakeEval5 = evaluation(bertYake[:5], answer_key[i],i)
    bertKeyEval5 = evaluation(bertKey[:5], answer_key[i],i)
    bertSelfEval5 = evaluation(bertSelf[:5], answer_key[i],i)
#    bertTopicEval5 = evaluation(bertTopic[:5], answer_key[i],i)

    
#     keybertEval3 = evaluation([x[0] for x in keyBertWords[i]][:3], answer_key[i], i)
#     selfTFIDFEval3 = evaluation([x[0] for x in ranked[i]][:3], answer_key[i], i)
#     yakeEval3 = evaluation([x[0] for x in yakeKeyWords[i]][:3], answer_key[i], i)
#     topicRankEval3 =evaluation(topicRankKeyWords[i][:3], answer_key[i], i)
#     tfidfLibEval3 = evaluation([x[0] for x in tfidfKeyphrases[i]][:3], answer_key[i],i)
    bertYakeEval3 = evaluation(bertYake[:3], answer_key[i],i)
    bertKeyEval3 = evaluation(bertKey[:3], answer_key[i],i)
    bertSelfEval3 = evaluation(bertSelf[:3], answer_key[i],i)
#    bertTopicEval3 = evaluation(bertTopic[:3], answer_key[i],i)

    
#     keybertEval1 = evaluation([x[0] for x in keyBertWords[i]][:1], answer_key[i], i)
#     selfTFIDFEval1 = evaluation([x[0] for x in ranked[i]][:1], answer_key[i], i)
#     yakeEval1 = evaluation([x[0] for x in yakeKeyWords[i]][:1], answer_key[i], i)
#     topicRankEval1 =evaluation(topicRankKeyWords[i][:1], answer_key[i], i)
#     tfidfLibEval1 = evaluation([x[0] for x in tfidfKeyphrases[i]][:1], answer_key[i],i)
    bertYakeEval1 = evaluation(bertYake[:1], answer_key[i],i)
    bertKeyEval1 = evaluation(bertKey[:1], answer_key[i],i)
    bertSelfEval1 = evaluation(bertSelf[:1], answer_key[i],i)
#    bertTopicEval1 = evaluation(bertTopic[:1], answer_key[i],i)

    
    
    
    
    

    if bertYakeEval >= 0:
        evalBertYake10.append(bertYakeEval)
        
    if bertKeyEval10 >= 0:
        evalBertKey10.append(bertKeyEval10)
        
    if bertSelfEval10 >= 0:
        evalSelfKey10.append(bertSelfEval10)
        

    if bertYakeEval7 >= 0:
        evalBertYake7.append(bertYakeEval7)
    if bertKeyEval7 >= 0:
        evalBertKey7.append(bertKeyEval7)
    if bertSelfEval7 >= 0:
        evalSelfKey7.append(bertSelfEval7)
        

    if bertYakeEval5 >= 0:
        evalBertYake5.append(bertYakeEval5)
    if bertKeyEval5 >= 0:
        evalBertKey5.append(bertKeyEval5)
    if bertSelfEval5 >= 0:
        evalSelfKey5.append(bertSelfEval5)

    if bertYakeEval3 >= 0:
        evalBertYake3.append(bertYakeEval3)
    if bertKeyEval3 >= 0:
        evalBertKey3.append(bertKeyEval3)
    if bertSelfEval3 >= 0:
        evalSelfKey3.append(bertSelfEval3)
        

    if bertYakeEval1 >= 0:
        evalBertYake1.append(bertYakeEval1)
    if bertKeyEval1 >= 0:
        evalBertKey1.append(bertKeyEval1)
    if bertSelfEval1 >= 0:
        evalSelfKey1.append(bertSelfEval1)
        
#    if bertTopicEval1 >= 0:
  
print("BertYake:")
print(average(evalBertYake10))  
print("BertKey:")
print(average(evalBertKey10))
# print("BertSelf:")
# print(average(evalBertSelf10))
#print(average(evalBertTopic10))




print("\n7 keywords")

print("BertYake:")
print(average(evalBertYake7))
print("BertKey:")
print(average(evalBertKey7))
# print("BertSelf:")
# print(average(evalBertSelf7))
#print(average(evalBertTopic7))
                          
    
    
print("\n5 keywords")
   
print("BertYake:")
print(average(evalBertYake5))
print("BertKey:")
print(average(evalBertKey5))
# print("BertSelf:")
# print(average(evalBertSelf5))
#print(average(evalBertTopic5))
                          
    
    
print("\n3 keywords")

print("BertYake:")
print(average(evalBertYake3))  
print("BertKey:")
print(average(evalBertKey3))
# print("BertSelf:")
# print(average(evalBertSelf3))
#print(average(evalBertTopic3))
                          
    
print("\n1 keywords")
   
print("BertYake:")
print(average(evalBertYake1))  
print("BertKey:")
print(average(evalBertKey1))
# print("BertSelf:")
# print(average(evalBertSelf1))
#print(average(evalBertTopic1))

0
new
[('uploadifi', 0.0), ('jpeg', 0.0), ('jpg', 0.0), ('png', 0.0), ('mime', 0.0)]
new
[('quit', 0.3229), ('vim', 0.0), ('window', 0.0), ('firefox', 0.0), ('ctrl', 0.0)]
new
[('matrix', inf), ('column', 0.2517), ('datafram', 0.0), ('error', 0.0), ('matlab', 0.0)]
new
[('href', 0.4013), ('string', 0.3708), ('winform', 0.3282), ('url', 0.0), ('search', 0.0)]
new
[('display', 0.0), ('function', 0.0), ('modifi', 0.0), ('whoi', 0.0), ('contact', 0.0)]
new
[('webpag', inf), ('browser', 0.2659), ('configur', 0.0), ('jmeter', 0.0), ('proxi', 0.0)]
new
[('chart', inf), ('imag', 0.0), ('coreplot', 0.0), ('barplot', 0.0), ('draw', 0.0)]
new
[('createhttp', 0.3684), ('httpwebrequest', 0.3387), ('feed', 0.0), ('fetch', 0.0), ('xml', 0.0)]
new
[('json', 0.0), ('code', 0.0), ('serial', 0.0), ('javascript', 0.0), ('gener', 0.0)]
new
[('imposs', 0.0), ('concaten', 0.0), ('inlin', 0.0), ('sql', 0.0), ('procedur', 0.0)]
new
[('protect', inf), ('decompil', 0.0), ('code', 0.0), ('reflector', 0.0), ('obfu

new
[('click', 0.3734), ('handler', 0.0), ('clickhand', 0.0), ('jqueri', 0.0), ('mobil', 0.0)]
new
[('data', 0.3493), ('timestamp', 0.2955), ('detect', 0.2858), ('sequenc', 0.0), ('event', 0.0)]
new
[('student', 6.413261189934106e+54), ('login', 0.0), ('userid', 0.0), ('session', 0.0), ('null', 0.0)]
new
[('error', 0.3342), ('href', 0.2775), ('widget', 3e-323), ('profil', 0.0), ('facebook', 0.0)]
new
[('code', 0.2852), ('keystrok', 0.0), ('command', 0.0), ('jsfiddl', 0.0), ('safari', 0.0)]
new
[('var', 0.0), ('vari', 0.0), ('varii', 0.0), ('index', 0.0), ('code', 0.0)]
new
[('command', 0.4134), ('programmat', 0.3039), ('mercuri', 0.0), ('directori', 0.0), ('commit', 0.0)]
new
[('hash', 0.3613), ('hashlib', 0.0), ('python', 0.0), ('librari', 0.0), ('encrypt', 0.0)]
new
[('cmake', 0.0), ('program', 0.0), ('creat', 0.0), ('cmakelist', 0.0), ('txt', 0.0)]
new
[('ssh', inf), ('sshd', 0.0), ('httpd', 0.0), ('server', 0.0), ('secur', 0.0)]
new
[('resolv', 2e-323), ('version', 0.0), ('error', 

IndexError: string index out of range

In [ ]:
for i in range(len(answer_key)):
  
    if i % 500 == 0:
        print(i)
    evalKeyBertYake = []
    evalKeyBertTFIDF = []
    evalKeyBertTR = []

    keyBertYake = [newTry(keyBertWords[i], yakeKeyWords[i]) for i in range(len(ranked))]
    keyBertYake = [x[0] for x in keyBertYake]
    keyBertYake = list(Counter(keyBertYake))[:10]

    
    keyBertTFIDF = [newTry(keyBertWords[i], ranked[i]) for i in range(len(ranked))]
    keyBertTFIDF = [x[0] for x in keyBertTFIDF]
    keyBertTFIDF = list(Counter(keyBertTFIDF))[:10]


    #keyBertTR = [newTry(keyBertWords[i], topicRankKeyWords[i]) for i in range(len(keyBertWords))]
    #keyBertTR = [x[0] for x in keyBertTR]
    #keyBertTR = list(Counter(keyBertTR))[:10]
    keyBertYakeEval = evaluation(keyBertYake, answer_key[i], i)
    keyBertTFIDFEval = evaluation(keyBertTFIDF, answer_key[i], i)
    #keyBertTREval = evaulation([x[0] for x in keyBertTR[i]][:10], answer_key[i], i)

    if keyBertYakeEval >= 0:
        evalKeyBertYake.append(keyBertYakeEval)
    if keyBertTFIDFEval >= 0:
        evalKeyBertTFIDF.append(keyBertTFIDFEval)
    #if keyBertTREval >= 0:
    #    evalKeyBertTFIDF.append(keyBertTREval)
    
    
print("KeyBERT + Yake:" + str(average(evalKeyBertYake)))
print("KeyBERT + TFIDF:" + str(average(evalKeyBertTFIDF)))
#print("KeyBERT + Topic Rank:" + str(average(evalKeyBertTR)))

    